In [ ]:
#Scrape and download all links to the local file system due to http speed limit
import urllib.request
html_doc = ''
def init():
    with urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_Greek_mythological_figures') as response:
        html_doc = response.read()

In [ ]:
from bs4 import BeautifulSoup

f = open("wiki/Greek-index.html", 'r')
html_doc =  f.read()
soup = BeautifulSoup(html_doc, 'html.parser')
all_links = soup.find_all('a')
print(len(all_links))

In [ ]:
#Download all of the relevant web pages from all the links
def download_pages():
    counter = 0
    for link in all_links:
        if(link.has_attr('href') and "/wiki/" in link['href']):
            if counter/len(all_links) % 10 == 0:
                print(str(counter/len(all_links) * 100) + '%')
            href = link['href']
            if '.jpg' or '.svg' or '.png' in href:
                continue
            urllib.request.urlretrieve("https://en.wikipedia.org"+href, href[href.rfind('/')+1:] + '.html')
    href = "/wiki/List_of_Greek_mythological_figures"
    urllib.request.urlretrieve("https://en.wikipedia.org"+href, href[href.rfind('/')+1:] + '.html')

In [ ]:
soup.h3.span.text
soup.h3.next_sibling
sibling = soup.h3
counter = 0
h3_tag = soup.h3
h3_sibling_content = soup.h3.next_sibling.next_sibling

#Go back 1 sibling to be able to grab the first h3 tag into the list
clean_siblings = [s for s in h3_tag.previous_sibling.next_siblings if (str(s).strip() != "" and (s.name == "h3" or s.name == "div" or s.name == "table" or s.name=="ul"))]
clean_siblings = clean_siblings[:-7]
clean_siblings.remove(clean_siblings[5])

it = iter(clean_siblings)
heading_content_dict= {}
for sibling in it:
    heading_content_dict[sibling.text.replace("[edit]","").replace("/"," & ")] = next(it)
print(len(heading_content_dict))
#print(heading_content_dict)
print(heading_content_dict.keys())
        
    #print(next(it))

In [ ]:
master_dict = {}

In [ ]:
import re

doc = heading_content_dict["Major gods and goddesses"]
soup_major = BeautifulSoup(str(doc), 'html.parser')
values = soup_major.find_all("td")
sub_pages = []

prime_list = []

major_mappings_uri = {} #uri to lowercase name
major_mappings_name = {} #lowercase name to uri
major_parents = {}
major_children = {}
for td in values:
    if not td.a.img:
        print(td.a['href'])
        f = open(td.a['href'][1:] + ".html", 'r')
        sub_doc = f.read()
        soup_major_sub = BeautifulSoup(str(sub_doc), 'html.parser')
        box_info = soup_major_sub.find_all("tr")
        
        vr = td.a['href']
        major_mappings_uri[td.a['href']] = vr[vr.rfind("/") + 1:].lower()
        
        major_mappings_name[vr[vr.rfind("/") + 1:].lower()] = td.a['href']
        
        for tr in box_info:
            if tr.th and (tr.th.text == "Children" or tr.th.text == "Parents" or tr.th.text == "Offspring"):
                print(tr.th.text + ":")
                st = re.sub(r'\s(or|and)\s', ',',tr.td.text) #Get rid of 'or' 'and' stop words
                #st = tr.td.text
                result = [x.strip().lower() for x in re.sub(r'(\[|\().*?(\]|\))','',st).split(",") if x != '']
                if tr.th.text == "Children" or tr.th.text == "Offspring" or tr.th.text == "Parents":
                    #master_dict.setdefault(vr[vr.rfind("/") + 1:].lower(), {})
                    if vr[vr.rfind("/") + 1:].lower() in master_dict:
                        master_dict[vr[vr.rfind("/") + 1:].lower()] = dict(master_dict[vr[vr.rfind("/") + 1:].lower()],**{str(tr.th.text).lower(): result})
                    else:
                        master_dict[vr[vr.rfind("/") + 1:].lower()] = {str(tr.th.text).lower(): result}
                        
                    if tr.th.text == "Children" or tr.th.text == "Offspring":
                        major_children[vr[vr.rfind("/") + 1:]] = result
                    elif tr.th.text == "Parents":
                        major_parents[vr[vr.rfind("/") + 1:]] = result
                #master_dict[vr[vr.rfind("/") + 1:].lower()] = {str(tr.th.text).lower(): result}
                print(result) #Parse out citation square brackets
        print()

In [ ]:
print(master_dict)

In [ ]:
import re

doc = heading_content_dict["Primordial deities"]
soup_prime = BeautifulSoup(str(doc), 'html.parser')
values = soup_prime.find_all("tr")

#links = []

prime_set = set()

prime_parents = {}
prime_children = {}

prime_uris = []
 
prime_mappings_uri = {}  #uri to name
prime_mappings_name = {} #name to uri
for tr in values:
    if tr.contents[3].a:
        f = open(tr.contents[3].a['href'][1:] + '.html','r')
        prime_uris.append(tr.contents[3].a['href'][1:])
        sub_doc = f.read()
        soup_prime_sub = BeautifulSoup(str(sub_doc), 'html.parser')
        box_info = soup_prime_sub.find_all("tr")
        print(tr.contents[3].a['href'])
        
        vr = re.sub(r'(\[|\().*?(\]|\))','',tr.contents[3].a['href']).strip().replace('_','')
        
        prime_mappings_uri[tr.contents[3].a['href']] = vr[vr.rfind("/") + 1:].lower()
        prime_mappings_name[vr[vr.rfind("/") + 1:].lower()] = tr.contents[3].a['href']
        
        prime_set.add(vr[vr.rfind("/") + 1:])
        for tr in box_info:
            if tr.th and (tr.th.text == "Children" or tr.th.text == "Parents" or tr.th.text == "Offspring"):
                st = re.sub(r'\s(or|and)\s', ',',tr.td.text) #Get rid of 'or' 'and' stop words
                #st = tr.td.text
                st = st.replace('with', '')
                result = [x.strip().lower() for x in re.split(r',|\n', re.sub(r'(\[|\().*?(\]|\))','',st)) if x != '']
                #v = re.sub(r'(\[|\().*?(\]|\))','',tr.contents[3].a['href']).replace('_','').strip()
               
                print(tr.th.text + ":")
                print(result) #Parse out citation square brackets
                
                if tr.th.text == "Children" or tr.th.text == "Offspring" or tr.th.text == "Parents":
                    #master_dict.setdefault()
                    if vr[vr.rfind("/") + 1:].lower() in master_dict:
                        master_dict[vr[vr.rfind("/") + 1:].lower()] = dict(master_dict[vr[vr.rfind("/") + 1:].lower()],**{str(tr.th.text).lower(): result})
                    else:
                        master_dict[vr[vr.rfind("/") + 1:].lower()] = {str(tr.th.text).lower(): result}
                        
                    if tr.th.text == "Children" or tr.th.text == "Offspring":
                        prime_children[vr[vr.rfind("/") + 1:]] = result
                    elif tr.th.text == "Parents":
                        prime_parents[vr[vr.rfind("/") + 1:]] = result

        print()

missing_parents = []
missing_children = []
for p in prime_set:
    if p not in prime_parents:
        #print("No Parent(s): " + p)
        missing_parents.append(p)
    if p not in prime_children:
        #print("No Children(s):" + p)
        missing_children.append(p)


        

In [ ]:
print(master_dict)
#print(master_dict['aphrodite'])
#from py2neo import Graph

In [ ]:
import unicodedata
def strip_accents(s):
   return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')

In [ ]:
import string
def fill_in_missing(uri, index, mapping_uri):
    parents_mapping = {} #current god to its parents
    children_mapping = {} #current god to its children
    incomplete_results = {}
    
    print("Uri: " + uri)
    f = open(uri[1:] + '.html','r')
    sub_doc = f.read()

    soup_t_sub = BeautifulSoup(str(sub_doc), 'html.parser')
    b = soup_t_sub.findAll("p")
        
    for i in b:
        st = re.sub(r'\<[^>]*\>' ,'',str(i))
        
        st = re.sub(r'(\[|\().*?(\]|\))','', st)
        translator = str.maketrans('', '', ",")
    
        st = st.translate(translator)
        st = strip_accents(st)
        k = re.search(r"(\b[A-Z]+\w*\b)\s*(?:\b[a-z]*\b\s)*(progeny|daughter|father|mother|son|offspring)(\b\w*\b\s)*(?:\s+of\s+)(?:\b\w*\b\s)?(\b[A-Z]+[a-z]+\b)(?:\sand\s)?(\b[A-Z]+\w*\b)?", st)

        #k = re.search(r"(\b[A-Z]+\w*\b)(?:\b[a-z]*\b\s)*(daughter|father|mother|son|offspring)(?:\b\w*\b\s)*(?:\s?of\s?)(?:\b[a-z]*\b\s)*(\b[A-Z]+\w+\b)(?:\sand\s)?(\b[A-Z]+\w*\b)?", st)
        #(daughter|father|mother|son)(?:\s?of\s?)(?:\b[a-z]*\b\s)*(\b[A-Z]+\w+\b)(?:\sand\s)?(\b[A-Z]+\w*\b\s)?
        if k:
            #print(k.group())
            print("???")
            print(uri in mapping_uri)
            print(str(k.group(1)).lower() == mapping_uri[uri].lower())
            print(str(k.group(1)).lower())
            print(mapping_uri[uri].lower())
            print("???")
            
            divinity_name = mapping_uri[uri].lower()
            
            if k.group(1).lower() == divinity_name:
                #wiki/Pallas
                #pallas daughter of _ and _
                if(k.group(2) in ('daughter','son', 'offspring','progeny')):
                #pallas mother of _ and _
                elif(k.group(2) in ('father','mother')):
 
            elif k.group(3).lower() == divinity_name or k.group(4).lower() == divinity_name:
                #x son of pallas and _
                #y father of pallas and _
                
            #if k.group(1).lower() == divinity_name
            #if(uri in mapping_uri and str(k.group(1)).lower() == mapping_uri[uri].lower()):
                #print(prime_mappings_name[k.group(1).lower()]) 
            if(k.group(2) in ('daughter','son', 'offspring','progeny')):                    
                #Who are my parents?
                if k.group(1).lower() == divinity_name and k.group(1) in parents_mapping:
                    parents_mapping[k.group(1)] = parents_mapping[k.group(1)].extend(k.group(3),k.group(4))
                else:
                    parents_mapping[k.group(1)] = [k.group(3), k.group(4)]
            elif(k.group(2) in ('father','mother')):
                #Who are my children?
                if k.group(1) in children_mapping:
                    children_mapping[k.group(1)] = children_mapping[k.group(1)].extend(k.group(3), k.group(4))
                else:
                    children_mapping[k.group(1)] = [k.group(3), k.group(4)]
            
            #else:
                #incomplete_results[uri] = [index, k]
                #print("Incomplete: " + uri + " : " + k.group(1))
                #print(k.group())
            print(k.group(0))
            print(k.group(1))
            print(k.group(2))
            print(k.group(3))
            print(k.group(4))
            #print("++++")            
    #print()
    return {"Parents": parents_mapping, "Children": children_mapping, "Incomplete": incomplete_results}

In [ ]:
#print(master_dict)
master_incomplete_results = []
def processIncompleteList(missingList, mapping_name, mapping_uri):
    results = {}
    incomplete_results = [] #Record those with incomplete results
    for idx, p in enumerate(missingList):
        print("Current Index: " + str(idx))
        results = fill_in_missing(mapping_name[p.lower()], idx, mapping_uri)
        print(results)
        if len(results["Incomplete"]) > 0:
            incomplete_results.append(results)
        if(len(results["Parents"]) > 0):
            if p.lower() in master_dict and "parents" in master_dict[p.lower()]:
                master_dict[p.lower()]["parents"] += list(results["Parents"].values())[0]
            else:
                master_dict[p.lower()] = {"parents": list(results["Parents"].values())[0]}
        if(len(results["Children"]) > 0):
            if p.lower() in master_dict and "children" in master_dict[p.lower()]:
                #print(master_dict[p.lower])
                master_dict[p.lower()]["children"] += list(results["Children"].values())[0]
            else:
                master_dict[p.lower()] = {"children": list(results["Children"].values())[0]}
        print()
    return incomplete_results
master_incomplete_results += processIncompleteList(missing_parents, prime_mappings_name, prime_mappings_uri)

In [ ]:
print(master_dict)

In [ ]:
print(master_incomplete_results)

In [ ]:
#for idx, p in enumerate(missing_children):
#    print("Current Index: " + str(idx))
#    results = fill_in_missing(prime_mappings_name[p.lower()], idx)
#    print(results)
#    if len(results["Incomplete"]) > 0:
#        incomplete_results.append(results)
#    print()

In [ ]:
master_incomplete_results += processIncompleteList(missing_children, prime_mappings_name, prime_mappings_uri)

In [ ]:
#Ignore Ourea
#Tatarus should be father of Giants
for i in master_incomplete_results:
    print(i)
tatarus = master_incomplete_results[0] #Father of the Giants
achlys = master_incomplete_results[5] #Daughter of Nyx

master_dict['tartarus']['children'] = ["giants"]
master_dict['achlys']['parents'].append('nyx')
#master_incomplete_results.clear() #discard rest of the results

In [ ]:
print(prime_mappings_name)

In [ ]:
print(major_mappings_name)

In [ ]:
print(prime_mappings_uri)

In [ ]:
print(len(master_dict))
print(master_dict)

In [ ]:
doc = heading_content_dict["Titans and Titanesses"]
soup_titan = BeautifulSoup(str(doc), 'html.parser')
values = soup_titan.find_all("tr")
#print(values[2:])
values = values[2:]

titan_set = set()

titan_parents = {}
titan_children = {}

titan_uris = []
 
titan_mappings_uri = {}  #uri to name
titan_mappings_name = {} #name to uri
for tr in values:
    #print(tr.contents)
    if len(tr.contents) < 4:
        continue
    if tr.contents[3].a:
        f = open(tr.contents[3].a['href'][1:] + '.html','r')
        titan_uris.append(tr.contents[3].a['href'][1:])
        sub_doc = f.read()
        soup_titan_sub = BeautifulSoup(str(sub_doc), 'html.parser')
        box_info = soup_titan_sub.find_all("tr")
        print(tr.contents[3].a['href'])
        
        vr = re.sub(r'(\[|\().*?(\]|\))','',tr.contents[3].a['href']).strip().replace('_','')
        
        titan_mappings_uri[tr.contents[3].a['href']] = vr[vr.rfind("/") + 1:].lower()
        titan_mappings_name[vr[vr.rfind("/") + 1:].lower()] = tr.contents[3].a['href']
        
        titan_set.add(vr[vr.rfind("/") + 1:])
        for tr in box_info:
            if tr.th and (tr.th.text == "Children" or tr.th.text == "Parents" or tr.th.text == "Offspring"):
                st = re.sub(r'\s(or|and)\s', ',',tr.td.text) #Get rid of 'or' 'and' stop words
                #st = tr.td.text
                st = st.replace('with', '')
                result = [x.strip().lower() for x in re.split(r',|\n', re.sub(r'(\[|\().*?(\]|\))','',st)) if x != '']
                #v = re.sub(r'(\[|\().*?(\]|\))','',tr.contents[3].a['href']).replace('_','').strip()
               
                print(tr.th.text + ":")
                print(result) #Parse out citation square brackets
                
                if tr.th.text == "Children" or tr.th.text == "Offspring" or tr.th.text == "Parents":
                    #master_dict.setdefault()
                    if vr[vr.rfind("/") + 1:].lower() in master_dict:
                        master_dict[vr[vr.rfind("/") + 1:].lower()] = dict(master_dict[vr[vr.rfind("/") + 1:].lower()],**{str(tr.th.text).lower(): result})
                    else:
                        master_dict[vr[vr.rfind("/") + 1:].lower()] = {str(tr.th.text).lower(): result}
                        
                    if tr.th.text == "Children" or tr.th.text == "Offspring":
                        titan_children[vr[vr.rfind("/") + 1:]] = result
                    elif tr.th.text == "Parents":
                        titan_parents[vr[vr.rfind("/") + 1:]] = result

        print()

missing_parents = []
missing_children = []
for p in titan_set:
    if p not in titan_parents:
        #print("No Parent(s): " + p)
        missing_parents.append(p)
    if p not in titan_children:
        #print("No Children(s):" + p)
        missing_children.append(p)


        

In [ ]:
print(missing_parents)
print(missing_children)

In [ ]:
result = processIncompleteList(missing_parents, titan_mappings_name, titan_mappings_uri)

In [ ]:
titan_mappings_name['selene']
titan_mappings_uri['/wiki/Selene']

In [ ]:
#if(uri in mapping_uri and str(k.group(1)).lower() == mapping_uri[uri].lower()):
print(list(result[-2]["Incomplete"].values())[0][1].group(1))
print(titan_mappings_uri['/wiki/Selene'].lower())
list(result[-2]["Incomplete"].values())[0][1].group(1).lower() == titan_mappings_uri['/wiki/Selene'].lower()

In [ ]:
print(len(result))
for i in result:
    print(i)